# **Food Commodity Prices Analysis and Forecasting in Kenya**

**By:** Charles Kagwanja   **|**   Kevin Kagia  |   Lucy Njambi  | 
 Mwenda Mugambi



---

In [1]:
# Importing necessary Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading the dataset
data = pd.read_csv('Data/wfp_food_prices_ken.csv')

# Displaying the first few rows of the dataset
data.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd
1,2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,cereals and tubers,Maize (white),90 KG,actual,Wholesale,KES,1480.0,20.5041
2,2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,pulses and nuts,Beans,KG,actual,Wholesale,KES,33.63,0.4659
3,2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,pulses and nuts,Beans (dry),90 KG,actual,Wholesale,KES,3246.0,44.9705
4,2006-01-15,Eastern,Kitui,Kitui,-1.366667,38.016667,cereals and tubers,Maize (white),KG,actual,Retail,KES,17.0,0.2355


In [3]:
# Dropping the first Row
data = data.drop(index=0)

# Descriptive statistics and dataset size
data.describe(include='all', datetime_is_numeric=True)

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
count,15735,15735,15735,15735,15735,15735,15735,15735,15735,15735,15735,15735,15735,15735
unique,212,7,22,62,62,62,8,47,14,2,2,1,6875,11386
top,2021-04-15,Rift Valley,Nairobi,Nairobi,-1.283333,36.816667,cereals and tubers,Maize (white),KG,actual,Wholesale,KES,10.0,0.0917
freq,600,5862,3516,1763,1763,1763,7435,1785,6429,10649,8454,15735,346,34


In [4]:
# Checking for missing values
data.isnull().sum()

date         0
admin1       0
admin2       0
market       0
latitude     0
longitude    0
category     0
commodity    0
unit         0
priceflag    0
pricetype    0
currency     0
price        0
usdprice     0
dtype: int64

In [5]:
# Unique values for certain columns
unique_admin1 = data['admin1'].nunique()
unique_markets = data['market'].nunique()
unique_commodities = data['commodity'].nunique()
unique_categories = data['category'].nunique()

print(f"unique_admin1 = {unique_admin1}\n" 
      f"unique_markets = {unique_markets}\n"
      f"unique_commodities = {unique_commodities}\n"
      f"unique_categories = {unique_categories}\n" )

unique_admin1 = 7
unique_markets = 62
unique_commodities = 47
unique_categories = 8



In [6]:
# Parsing the date column to datetime
data['date'] = pd.to_datetime(data['date'], format='%Y/%m/%d')

# Ensuring the data is sorted by date for time series analysis
data.sort_values(by='date', inplace=True)

# Resetting index after sorting
data.reset_index(drop=True, inplace=True)

# Checking the datatype of each column after conversion and sorting
data_types = data.dtypes

# Displaying the first few rows after data preparation
prepared_data_head = data.head()

data_types, prepared_data_head

(date         datetime64[ns]
 admin1               object
 admin2               object
 market               object
 latitude             object
 longitude            object
 category             object
 commodity            object
 unit                 object
 priceflag            object
 pricetype            object
 currency             object
 price                object
 usdprice             object
 dtype: object,
         date       admin1       admin2                      market  latitude  \
 0 2006-01-15        Coast      Mombasa                     Mombasa     -4.05   
 1 2006-01-15  Rift Valley  Uasin Gishu  Eldoret town (Uasin Gishu)  0.516667   
 2 2006-01-15  Rift Valley  Uasin Gishu  Eldoret town (Uasin Gishu)  0.516667   
 3 2006-01-15  Rift Valley  Uasin Gishu  Eldoret town (Uasin Gishu)  0.516667   
 4 2006-01-15  Rift Valley  Uasin Gishu  Eldoret town (Uasin Gishu)  0.516667   
 
    longitude            category      commodity   unit priceflag  pricetype  \
 0  39.666

In [7]:
# Renaming the 'admin1' and 'admin2' columns to more understandable names
data_renamed = data.rename(columns={'admin1': 'region', 'admin2': 'district'})

# Display the first few rows to confirm the changes
data_renamed.head()

,date,region,district,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,cereals and tubers,Maize (white),90 KG,actual,Wholesale,KES,1480.0,20.5041
1,2006-01-15,Rift Valley,Uasin Gishu,Eldoret town (Uasin Gishu),0.516667,35.283333,pulses and nuts,Beans (dry),90 KG,actual,Wholesale,KES,2799.0,38.7777
2,2006-01-15,Rift Valley,Uasin Gishu,Eldoret town (Uasin Gishu),0.516667,35.283333,pulses and nuts,Beans,KG,actual,Wholesale,KES,45.85,0.6353
3,2006-01-15,Rift Valley,Uasin Gishu,Eldoret town (Uasin Gishu),0.516667,35.283333,cereals and tubers,Maize (white),90 KG,actual,Wholesale,KES,1192.0,16.5141
4,2006-01-15,Rift Valley,Uasin Gishu,Eldoret town (Uasin Gishu),0.516667,35.283333,cereals and tubers,Maize,KG,actual,Wholesale,KES,12.96,0.1796


In [8]:
# Parsing the date column to datetime and sorting the dataset by date
data_renamed['date'] = pd.to_datetime(data_renamed['date'], format='%d/%m/%Y')
data_renamed.sort_values(by='date', inplace=True)

# Selecting 'Maize (white)' commodity data for the Modeling phase
selected_commodity = 'Maize (white)'
maize_data = data_renamed[data_renamed['commodity'] == selected_commodity]

# Counting the number of records available for each market and price type combination for 'Maize (white)'
maize_market_counts = maize_data.groupby(['market', 'pricetype']).size().reset_index(name='count')

# Displaying the combinations with the highest number of records
maize_market_counts.sort_values(by='count', ascending=False).head(10)

,market,pricetype,count
6,Kitui,Retail,180
8,Mandera,Retail,179
7,Lodwar (Turkana),Retail,177
10,Marsabit,Retail,171
0,Eldoret town (Uasin Gishu),Wholesale,170
12,Nairobi,Wholesale,170
11,Mombasa,Wholesale,167
5,Kisumu,Wholesale,164
4,Kilifi,Retail,71
3,Kajiado,Retail,70


Based on the availability of data for 'Maize (white)', the following market and price type combinations have the highest number of records:

* Kitui - Retail (180 records)
* Mandera - Retail (179 records)
* Lodwar (Turkana) - Retail (177 records)
* Marsabit - Retail (171 records)
* Eldoret town (Uasin Gishu) - Wholesale (170 records)
* Nairobi - Wholesale (170 records)

In [9]:
# Step 1: Filtering data for 'Maize (white)' in Kitui market with Retail prices
selected_market = 'Kitui'
selected_pricetype = 'Retail'

baseline_data = maize_data[(maize_data['market'] == selected_market) & 
                           (maize_data['pricetype'] == selected_pricetype)]

# Step 2: Splitting the data into training and test sets
train_data, test_data = train_test_split(baseline_data, test_size=0.2, shuffle=False)

# Step 3: Implementing the Naive Forecasting Model
naive_forecast = np.array(train_data['price'][1:])  # Excluding the first point which has no prior point

# Actual values to compare with excluding the last point which has no next point
actual_values = np.array(train_data['price'][:-1])

# Step 4: Evaluating the model
mae_naive = mean_absolute_error(actual_values, naive_forecast)

mae_naive

2.024195804195804

### Baseline Model Evaluation
The Naive Forecasting Model for 'Maize (white)' prices in the Kitui market (Retail prices) has been evaluated. The Mean Absolute Error (MAE) for this baseline model is approximately 2.02 KES. This metric represents the average magnitude of errors in a set of predictions, without considering their direction.